---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#% matplotlib inline
#import seaborn as sns
#import warnings
#warnings.filterwarnings("ignore")
#pd.set_option('display.max_columns',100)
#from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.preprocessing import LabelEncoder
#from sklearn.metrics import accuracy_score, roc_curve, auc

In [93]:
# Loading the main datasets to be used
#train_data = pd.read_csv('readonly/train.csv', encoding='cp1252')
#test_data = pd.read_csv('readonly/test.csv', encoding='cp1252')
#addresses_data = pd.read_csv('readonly/addresses.csv', encoding='cp1252')
#latlons_data = pd.read_csv('readonly/latlons.csv', encoding='cp1252')

In [94]:
#train_data.head(1)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,non_us_str_code,country,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,60606,NaN,USA,2004-03-16 11:40:00,2005-03-21 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0


In [95]:
#  To know the number of rows and columns in the dataset
#train_data.shape

(250306, 34)

In [96]:
# To find the type of each feature, and whether there are any missing data or not
#train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 34 columns):
ticket_id                     250306 non-null int64
agency_name                   250306 non-null object
inspector_name                250306 non-null object
violator_name                 250272 non-null object
violation_street_number       250306 non-null float64
violation_street_name         250306 non-null object
violation_zip_code            0 non-null float64
mailing_address_str_number    246704 non-null float64
mailing_address_str_name      250302 non-null object
city                          250306 non-null object
state                         250213 non-null object
zip_code                      250305 non-null object
non_us_str_code               3 non-null object
country                       250306 non-null object
ticket_issued_date            250306 non-null object
hearing_date                  237815 non-null object
violation_code                250306 non-null obj

In [97]:
# To find out if there is any duplicated rows to drop
#train_data.duplicated().any().sum()

0

In [98]:
# Dropping Null values of compliance, so that we can only evaluate yes or no values
#train_data = train_data[train_data['compliance'].notnull()]

In [99]:
#train_data.shape

(159880, 34)

In [100]:
# We can find out that there are three features that only have 0, 1 and 3 data respectively
# (violation_zip_code, grafitti_status, non_us_str_code)
# We also have 2 features that have less than 20 % of the data which are (payment_date, collection_status)
# Dropping columns that are not used because they are either descrioptions or make no sense in prediction
#features = ['country', 'city','disposition', 'violation_code', 'judgment_amount',
            #'ticket_issued_date', 'hearing_date']
#label = ['compliance']

In [104]:
def payment_duration(df):
    '''
        Enter the dataframe that have the proposed features which you want to convert
        Converting the duration from issuing ticket to expected hearing to days    
                                                                                '''
    # Converting the dates to time periods to deal with it
    df['duration_to_pay'] = pd.to_datetime(df['hearing_date']) - pd.to_datetime(df['ticket_issued_date'])
    # Finding the period required for payment in days
    df['duration_to_pay'] = pd.to_timedelta(df['duration_to_pay']).dt.days
    # Mapping the duration into categorical values
    df['duration_to_pay'] = np.select([df['duration_to_pay'] < 90, df['duration_to_pay'] < 180,
                                          df['duration_to_pay'] < 270, df['duration_to_pay'] <365],
                                         ['one_quarter', 'two_quarters', 'three_quarters', 'year'],
                                         default = 'more_than_year')
    
    df['judgment_amount'] = np.select([df['judgment_amount'] < 100, df['judgment_amount'] < 200,
                                        df['judgment_amount'] < 300, df['judgment_amount'] < 400,
                                        df['judgment_amount'] < 500],
                                         ['<100', '<200', '<300', '<400', '<500'],
                                         default = '>500')
    # Dropping the columns that are no longer necessary
    df.drop(['ticket_issued_date', 'hearing_date'], axis = 1, inplace = True)

    return (df)

In [102]:
# Setting index for both datasets as of the ticket id
train_data.set_index('ticket_id', inplace = True)
test_data.set_index('ticket_id', inplace = True)

In [105]:
# Finding a brief summary about statistical realtionship between numerical features
X = train_data[features]
# Using a pre-defined function
X = payment_duration(X)
# Dataset in its final shape
X.head()

,country,city,disposition,violation_code,judgment_amount,duration_to_pay
ticket_id,,,,,,
22056,USA,CHICAGO,Responsible by Default,9-1-36(a),<400,more_than_year
27586,USA,Detroit,Responsible by Determination,61-63.0600,>500,more_than_year
22046,USA,LOG BEACH,Responsible by Default,9-1-36(a),<400,year
18738,USA,Clinton,Responsible by Default,61-63.0500,>500,three_quarters
18735,USA,Detroit,Responsible by Default,61-63.0100,<200,three_quarters


In [106]:
# The column to predict
y = train_data[label]
y.head()

,compliance
ticket_id,
22056,0.0
27586,1.0
22046,0.0
18738,0.0
18735,0.0


In [107]:
def categorical_data(df):
    for i, col in zip(df.index, ['country','city', 'disposition', 'violation_code', 'disposition', 'judgment_amount', 'duration_to_pay']):
        df[col] = df[col].astype(str)
        le = LabelEncoder()
        le.fit(df[col])
        df[col] = le.transform(df[col])
    return(df)

In [108]:
# Doing the same convertion to test dataset
test = test_data[features]
test = payment_duration(test)
test.head()

,country,city,disposition,violation_code,judgment_amount,duration_to_pay
ticket_id,,,,,,
284932,USA,DETROIT,Responsible by Default,22-2-61,<300,one_quarter
285362,USA,DETROIT,Responsible by Default,22-2-88(b),>500,one_quarter
285361,USA,DETROIT,Responsible by Default,22-2-43,<200,one_quarter
285338,USA,WOODHAVEN,Responsible by Default,22-2-88(b),<300,one_quarter
285346,USA,LIVONIA,Responsible by Default,22-2-45,<200,one_quarter


In [109]:
# Using a pre-defined function
X = categorical_data(X)
test = categorical_data(test)

In [110]:
X.head()

,country,city,disposition,violation_code,judgment_amount,duration_to_pay
ticket_id,,,,,,
22056,4,543,2,143,3,0
27586,4,952,3,74,5,0
22046,4,1930,2,143,3,4
18738,4,725,2,73,5,2
18735,4,952,2,72,1,2


In [125]:
# Splitting the dataset with a test size of 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [127]:
# Identifying the classifier to be used
clf = RandomForestClassifier()
# Parameters to try to find the best through them using gridsearch
param_grid = {'min_samples_split':[5, 10, 15], 'max_depth':[5,10,15]}
# Using grid search to find the best parameters relating to the score of AUC
grid_search = GridSearchCV(clf, param_grid, scoring = 'roc_auc')
# since grid search would require the ytrain shape to be yshape of n_samples
c, r = y_train.shape
y_train = y_train.values.reshape(c,)
# Fitting the model created
grid_search.fit(X_train, y_train)
# Predicting the trained data 
y_train_predicted = grid_search.predict(X_train)
# Predicting the test data
y_predicted = grid_search.predict(X_test)
y_probs = grid_search.predict_proba(X_test)[:, 1]
# False positive ratio and True positive ratio
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
# Finding the area under the curve accuracy
print('auc is: {}' .format(auc(fpr,tpr)))

auc is: 0.7948891078334583


In [128]:
grid_search.best_score_, grid_search.best_params_

(0.80280796821443989, {'max_depth': 10, 'min_samples_split': 5})

In [133]:
test_y_probs = grid_search.predict_proba(test)[:, 1]

In [137]:
test['compliance'] = test_y_probs
test['compliance'].head()

,country,city,disposition,violation_code,judgment_amount,duration_to_pay,compliance
ticket_id,,,,,,,
284932,0,739,5,31,2,1,0.523964
285362,0,739,5,48,5,1,0.424886
285361,0,739,5,19,1,1,0.311226
285338,0,3040,5,48,2,1,0.487193
285346,0,1558,5,21,1,1,0.449215
285345,0,1558,5,48,2,1,0.474810
285347,0,1558,5,96,0,1,0.412131
285342,0,2285,6,48,2,1,0.519989
285530,0,1084,5,48,5,1,0.193102


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',100)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_curve, auc

def blight_model():

    # Loading the datasets to be used
    train_data = pd.read_csv('readonly/train.csv', encoding='cp1252')
    test_data = pd.read_csv('readonly/test.csv', encoding='cp1252')
    
    # Dropping Null values of compliance, so that we can only evaluate yes or no values
    train_data = train_data[train_data['compliance'].notnull()]

    # We can find out that there are three features that only have 0, 1 and 3 data respectively
    # (violation_zip_code, grafitti_status, non_us_str_code)
    # We also have 2 features that have less than 20 % of the data which are (payment_date, collection_status)
    # Dropping columns that are not used because they are either descrioptions or make no sense in prediction
    features = ['country', 'city', 'state', 'zip_code','disposition', 'violation_code', 'judgment_amount',
                'ticket_issued_date', 'hearing_date']
    label = ['compliance']

    # Defining a function to be used
    def payment_duration(df):
        '''
            Enter the dataframe that have the proposed features which you want to convert
            Converting the duration from issuing ticket to expected hearing to days    
                                                                                    '''
        # Converting the dates to time periods to deal with it
        df['duration_to_pay'] = pd.to_datetime(df['hearing_date']) - pd.to_datetime(df['ticket_issued_date'])
        # Finding the period required for payment in days
        df['duration_to_pay'] = pd.to_timedelta(df['duration_to_pay']).dt.days
        # Mapping the duration into categorical values
        df['duration_to_pay'] = np.select([df['duration_to_pay'] < 90, df['duration_to_pay'] < 180,
                                              df['duration_to_pay'] < 270, df['duration_to_pay'] <365],
                                             ['one_quarter', 'two_quarters', 'three_quarters', 'year'],
                                             default = 'more_than_year')
        # Dropping the columns that are no longer necessary
        df.drop(['ticket_issued_date', 'hearing_date'], axis = 1, inplace = True)

        return (df)

    # Defining a function to be used
    def categorical_data(df):
        for i, col in zip(df.index, ['country','city', 'state', 'zip_code', 'disposition', 'violation_code', 'disposition', 'duration_to_pay']):
            df[col] = df[col].astype(str)
            le = LabelEncoder()
            le.fit(df[col])
            df[col] = le.transform(df[col])
        return(df)

    # Setting index for both datasets as of the ticket id
    train_data.set_index('ticket_id', inplace = True)
    test_data.set_index('ticket_id', inplace = True)

    # Finding a brief summary about statistical realtionship between numerical features
    X = train_data[features]
    # Using a pre-defined function
    X = payment_duration(X)
    y = train_data[label]

    # Doing the same convertion to test dataset
    test = test_data[features]
    test = payment_duration(test)

    # Using a pre-defined function
    X = categorical_data(X)
    test = categorical_data(test)

    # Splitting the dataset with a test size of 25%
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

    # Identifying the classifier to be used
    clf = RandomForestClassifier()
    # Parameters to try to find the best through them using gridsearch
    param_grid = {'min_samples_split':[5, 10, 15], 'max_depth':[5,10,15]}
    # Using grid search to find the best parameters relating to the score of AUC
    grid_search = GridSearchCV(clf, param_grid, scoring = 'roc_auc')
    # since grid search would require the ytrain shape to be yshape of n_samples
    c, r = y_train.shape
    y_train = y_train.values.reshape(c,)
    # Fitting the model created
    grid_search.fit(X_train, y_train)
    # Predicting the trained data 
    y_train_predicted = grid_search.predict(X_train)
    # Predicting the test data
    y_predicted = grid_search.predict(X_test)
    # Because auc assumes continues value of y_predicted not binary value
    y_probs = grid_search.predict_proba(X_test)[:, 1]
    # False positive ratio and True positive ratio
    fpr, tpr, thresholds = roc_curve(y_test, y_probs)
    # Finding the area under the curve accuracy
    print('The model cross validation auc is: {}' .format(auc(fpr,tpr)))
    print('best parameters for the model are: {}'.format(grid_search.best_params_))

    # Using the model on the test dataset
    test_y_probs = grid_search.predict_proba(test)[:, 1]
    test['compliance'] = test_y_probs
    
    return test['compliance']

In [4]:
blight_model()

The model cross validation auc is: 0.8058841350943301
best parameters for the model are: {'max_depth': 15, 'min_samples_split': 10}


ticket_id
284932    0.439703
285362    0.365100
285361    0.411893
285338    0.318448
285346    0.379374
285345    0.385114
285347    0.343136
285342    0.711281
285530    0.114444
284989    0.241384
285344    0.613071
285343    0.277794
285340    0.365100
285341    0.560512
285349    0.301990
285348    0.280000
284991    0.223091
285532    0.407428
285406    0.407428
285001    0.319827
285006    0.152803
285405    0.143697
285337    0.184722
285496    0.362460
285497    0.385114
285378    0.143697
285589    0.318929
285585    0.406389
285501    0.411893
285581    0.298433
            ...   
376367    0.299650
376366    0.331421
376362    0.363041
376363    0.396601
376365    0.366317
376364    0.251421
376228    0.272551
376265    0.128761
376286    0.651984
376320    0.272551
376314    0.251421
376327    0.654545
376385    0.654545
376435    0.227428
376370    0.654545
376434    0.499085
376459    0.317115
376478    0.267392
376473    0.413934
376484    0.216127
376482    0.302868
37